Por ejemplo considere el caso de querer estimar, usando una variable gaussiana 1D, la distancia al siguiente auto en una ruta contando la cantidad de pixels en su silueta.

# Modelo de contingencia de las variables de estado del mundo $w$ sobre los datos $x$

$$
P(W \mid X)
$$
Dado que el model es univarial y continuo, eligimos la variable guassiana 1D, para ello fijamos la  $\sigma^2$ (varianza) (Porque??) y hacemos que $\mu$ este en funcion de los datos, asi tenemos dos variables nuevas:
$ \phi_0 + \phi_1 $
y la funcion de probabilidad queda como:
$$
Pr(w \mid x, \theta)
= \mathrm{Norm}_w\left[ \phi_0 + \phi_1 x,\, \sigma^2 \right]
$$
Por lo que los parametros del modelo son:
$$
\theta = \{ \phi_0,\ \phi_1,\ \sigma \}
$$
Esta formulacion se conoce como *Linear Regression*
El algoritmo de aprendizaje para este modelo enparda $\{ x_i,\ w_i \}_{i=1}^{I}$ los datos del dataset con los parametros del modelo.
El enfoque **MAP** seria:
$$
\hat{\theta}
= \underset{\theta}{\arg\max}\ \big[ Pr(\theta \mid w_{1\ldots I}, x_{1\ldots I}) \big]
$$

$$
= \underset{\theta}{\arg\max}\ \big[ Pr(w_{1\ldots I} \mid x_{1\ldots I}, \theta)\ Pr(\theta) \big]
\quad\text{(por Bayes)}
$$

$$
= \underset{\theta}{\arg\max}\ 
\left[
\prod_{i=1}^{I} Pr(w_i \mid x_i,\ \theta)\ Pr(\theta)
\right]
$$

# Aclaracion frase del libro

"where we have assumed that the I training pairs {x_i , w_i}_{i=1}^I are independent, and defined a suitable prior Pr(θ)."

A continuación se explica cada parte.

---

## 1. Independencia de los pares (x_i, w_i)

El dataset contiene pares:

(x_1, w_1), (x_2, w_2), ..., (x_I, w_I)

Asumir que son independientes significa que cada observación del dataset no depende de las demás. Esto permite factorizar la probabilidad conjunta de esta forma:

Pr(w_1,...,w_I | x_1,...,x_I, θ) = ∏_{i=1}^I Pr(w_i | x_i, θ)

Esta suposición se llama i.i.d. (independiente e idénticamente distribuido).

---

## 2. Definir un prior Pr(θ)

Los parámetros del modelo son:

θ = {φ_0, φ_1, σ}

En el enfoque bayesiano, estos parámetros tienen distribuciones previas antes de ver los datos. Por ejemplo:

φ_0 ~ N(0, 100)
φ_1 ~ N(0, 100)
σ ~ HalfNormal(5) -> Significa distribución Half-Normal con escala 5.

Este prior Pr(θ) representa nuestras creencias iniciales sobre los parámetros.

---

## 3. Cómo se combinan independencia + prior con Bayes

Usando la regla de Bayes:

Pr(θ | datos) ∝ Pr(datos | θ) * Pr(θ)

Y usando independencia:

Pr(datos | θ) = ∏_{i=1}^I Pr(w_i | x_i, θ)

Esto lleva a la expresión para MAP:

θ^ = argmax_θ [ ∏_{i=1}^I Pr(w_i | x_i, θ) * Pr(θ) ]

---

## Resumen

- Asumimos que cada ejemplo del dataset es independiente.
- Definimos un prior sobre los parámetros para hacer inferencia bayesiana.
- Al combinar prior y likelihood (producto por independencia), obtenemos la ecuación usada para estimación MAP.






# A continuacion se va a realizar un ejemplo con la base de datos de Kitti
1.- data_object_image_2.zip -> es la base de datos principal
2.- data_object_label_2.zip
3.- data_object_calib.zip
# Estrategia general para el ejercicio
1.- Detectar si la imagen contien o no contiene un auto
2.- Si contiene extraer la region de la imagen donde  esta el auto
3.- Mediante un proceso de background removal extraeer el contorno del auto
    - Vamos a usar una opcion mas realista y dura para obtener el contorno del auto una vez detectado, mediante la segmentación dentro del bbox (GrabCut / MaskRCNN / SAM, etc.)
    - Hay que validar el proceso anterior (Otra VA que diga si el contorno es valido o no)
    - Despues contamos pixeles del resultado
4.- Estimar la distancia del vehiculo usando Regression (El algoritmo de inferencia del libro)
5.- Que distancia?
    Distancia desde el centro óptico de la cámara al centro 3D del auto
    Porque eso es exactamente lo que representa z en KITTI Object Detection. Es la profundidad respecto a la cámara izquierda.
# Formato del archivo label de KITTI
```text
0:type
1:truncated
2:occluded
3:alpha
4:left
5:top
6:right
7:bottom
8:height
9:width
10:length
11:x
12:y
13:z     <--- distancia en metros
14:rotation_y
```

In [27]:
import tensorflow as tf
from pathlib import Path

def kitti_pairs_tf_datasets(
    root_abs_path,
    test_size=0.2,
    val_size=0.1,
    seed=42,
    shuffle=True,
):
    """
    Crea tres tf.data.Dataset (train, val, test) con pares (img_path, label_path).

    root_abs_path: str
        Path absoluto donde existen las carpetas image_2/ y label_2/

    test_size: float
        Porcentaje de test (0.2 = 20%)

    val_size: float
        Porcentaje de validation (0.1 = 10%)

    Uso típico en Jupyter:
        train_ds, val_ds, test_ds = kitti_pairs_tf_datasets("/ruta/kitti/training")
    """

    root = Path(root_abs_path).expanduser().resolve()
    img_dir = root / "image_2"
    label_dir = root / "label_2"

    # Verificar existencia
    if not img_dir.exists():
        raise FileNotFoundError(f"No existe {img_dir}")
    if not label_dir.exists():
        raise FileNotFoundError(f"No existe {label_dir}")

    # Listar imágenes
    img_paths = sorted([str(p) for p in img_dir.glob("*.png")])
    if len(img_paths) == 0:
        raise ValueError(f"No se encontraron imágenes PNG en {img_dir}")
    print(f"cantidad de imagenes cargadas: {len(img_paths)}")
    # Emparejar images - labels
    pairs = []
    for img_path in img_paths:
        frame_id = Path(img_path).stem
        label_path = label_dir / f"{frame_id}.txt"
        if label_path.exists():
            pairs.append((img_path, str(label_path)))

    if len(pairs) == 0:
        raise ValueError("No se encontraron pares imagen-label")

    # Convertir a tensores
    img_tensor = tf.constant([p[0] for p in pairs], dtype=tf.string)
    lab_tensor = tf.constant([p[1] for p in pairs], dtype=tf.string)

    n = tf.shape(img_tensor)[0]

    # Shuffle
    if shuffle:
        idx = tf.random.shuffle(tf.range(n), seed=seed)
        img_tensor = tf.gather(img_tensor, idx)
        lab_tensor = tf.gather(lab_tensor, idx)

    # Calcular tamaños
    n_test = int(n.numpy() * test_size)
    n_val  = int(n.numpy() * val_size)
    n_train = n.numpy() - n_test - n_val

    # Particiones
    train_imgs = img_tensor[:n_train]
    train_labs = lab_tensor[:n_train]

    val_imgs = img_tensor[n_train:n_train + n_val]
    val_labs = lab_tensor[n_train:n_train + n_val]

    test_imgs = img_tensor[n_train + n_val:]
    test_labs = lab_tensor[n_train + n_val:]

    # tf.data.Dataset
    train_ds = tf.data.Dataset.from_tensor_slices((train_imgs, train_labs))
    val_ds   = tf.data.Dataset.from_tensor_slices((val_imgs, val_labs))
    test_ds  = tf.data.Dataset.from_tensor_slices((test_imgs, test_labs))

    return train_ds, val_ds, test_ds


In [28]:
train_ds, val_ds, test_ds = kitti_pairs_tf_datasets("/home/operador/Documents/datasets/trainning")

cantidad de imagenes cargadas: 7481


Bien ahora sieguiendo con el libro, necesito crear un "detector de autos" pero siguiendo el ejercicio del libro:, por lo tanto es un problema de clasificacion y vamos a usar la distribucion categorical.
Como algorithmo de aprendizaje vamos a usar Maximum Likelihood primero.
Porque Para hacer la regression necesito saber si estoy lidiando con un auto o no, extraer los features que me pide el ejercicio (solo disponible si hay un auto , ademas debo saber donde esta, eso lo puedo aprender con el dataset que me dieron.) y recien ahi hacer la regresssion. Por eso que este ejercicio a pesar de que se trata de una regression tiene pasos escondididos que no son "puros" del ejericicio sino accesrios.
Para ello necesito extraer los siguientes features:
- área del contorno
- relación de aspecto
- ancho y alto
- perímetro
- compacidad
- posición del bounding box
Solo usando lo que se aprendio hasta el capitulo 6 (sin deep learning, CNN, HOG u otros metodos).
Como es *clasificacion Binaria* usamos Bernoulli:



# Modelo matemático para detección binaria de autos (regresión logística)

## 1. Notación

En KITTI, cada objeto anotado tiene un bounding box 2D definido por  
$(x_1, y_1, x_2, y_2)$.

A partir de ese bounding box definimos los **features**:

- $w_{bb} = x_2 - x_1$  (ancho del bounding box)
- $h_{bb} = y_2 - y_1$  (alto)
- $a_{bb} = w_{bb}\,h_{bb}$ (área)
- $r_{bb} = \dfrac{w_{bb}}{h_{bb}}$ (aspect ratio)

El vector de entrada es entonces:

$$
x =
\begin{bmatrix}
w_{bb} \\
h_{bb} \\
a_{bb} \\
r_{bb}
\end{bmatrix}
\in \mathbb{R}^4
$$

La etiqueta binaria se define como:

- $t = 1$ si el objeto es un **Car**
- $t = 0$ si es **Pedestrian**, **Cyclist**, u otro tipo

Los parámetros del modelo son:

- $w \in \mathbb{R}^4$
- $b \in \mathbb{R}$

Nuestro dataset es:

$$
\{ (x_n, t_n) \}_{n=1}^N
$$

---

## 2. Modelo probabilístico (Bernoulli + regresión logística)

Queremos modelar:

$$
p(t = 1 \mid x, w, b) = y(x)
$$

$$
p(t = 0 \mid x, w, b) = 1 - y(x)
$$

Modelo lineal + sigmoide:

$$
a = w^\top x + b
$$

$$
y(x) = \sigma(a) = \frac{1}{1 + \exp(-a)}
$$

Distribución Bernoulli:

$$
p(t \mid x, w, b) = y(x)^t \, (1 - y(x))^{1 - t}
$$

---

## 3. Verosimilitud y log-verosimilitud

Asumiendo datos independientes:

$$
L(w,b) = \prod_{n=1}^N p(t_n \mid x_n, w,b)
= \prod_{n=1}^N y_n^{t_n} (1 - y_n)^{1 - t_n}
$$

donde $y_n = \sigma(w^\top x_n + b)$.

Log-verosimilitud:

$$
\ell(w,b) =
\sum_{n=1}^N \Big[ t_n \log(y_n) + (1 - t_n)\log(1 - y_n) \Big]
$$

---

## 4. Por qué necesitamos el gradiente

La estimación por ML consiste en:

$$
(\hat{w}, \hat{b}) = \arg\max_{w,b}\, \ell(w,b)
$$

Para aplicar métodos de optimización (como gradient ascent) necesitamos los gradientes de $\ell$.

---

## 5. Derivación del gradiente

Resultado final para cada muestra:

$$
\nabla_w \ell_n(w,b) = (t_n - y_n)\, x_n
$$

Sumando sobre todas:

$$
\nabla_w \ell(w,b) = \sum_{n=1}^N (t_n - y_n)\, x_n
$$

Gradiente respecto de $b$:

$$
\frac{\partial \ell(w,b)}{\partial b} =
\sum_{n=1}^N (t_n - y_n)
$$

---

## 6. Método de aprendizaje e inferencia

### Nombre del método

**Regresión logística binaria con máxima verosimilitud**, optimizada mediante **gradient ascent**.

---

### 6.1. Actualización de parámetros

$$
w^{(k+1)} = w^{(k)} + \eta \sum_{n=1}^N (t_n - y_n) x_n
$$

$$
b^{(k+1)} = b^{(k)} + \eta \sum_{n=1}^N (t_n - y_n)
$$

---

### 6.2. Inferencia

Dado un nuevo vector de características `x_new`:

```
y_new = sigma( w_hat^T x_new + b_hat )
```

Regla de decisión:

- Si `y_new ≥ 0.5` → hay auto  
- Si `y_new < 0.5` → no hay auto

---

## 7. Resumen

- Modelo: regresión logística binaria  
- Distribución: Bernoulli  
- Aprendizaje: máxima verosimilitud  
- Gradiente:

$$
\nabla_w \ell(w,b) = \sum_{n=1}^N (t_n - y_n)x_n
$$

- Inferencia: $y_{\text{new}} = \sigma(\hat{w}^\top x_{\text{new}} + \hat{b})$ con umbral 0.5

Aclaracion:
```
statistical model like logistic regression, which predicts the probability of a categorical outcome
```

In [29]:
import tensorflow as tf

# ============================================
# 1) Extracción de features del bounding box
#    x = [w_bb, h_bb, area, aspect]
# ============================================

def kitti_bbox_features(x1, y1, x2, y2):
    """
    x1, y1, x2, y2 pueden ser escalares o tensores (por ejemplo (N,)).

    Devuelve un tensor de shape (..., 4) con:
        [width, height, area, aspect_ratio]
    """
    x1 = tf.convert_to_tensor(x1, dtype=tf.float32)
    y1 = tf.convert_to_tensor(y1, dtype=tf.float32)
    x2 = tf.convert_to_tensor(x2, dtype=tf.float32)
    y2 = tf.convert_to_tensor(y2, dtype=tf.float32)

    width  = x2 - x1          # w_bb
    height = y2 - y1          # h_bb
    area   = width * height   # a_bb

    # aspect ratio = width / height (evitar división por cero)
    eps = tf.constant(1e-6, dtype=tf.float32)
    aspect = width / (height + eps)

    # stack en el último eje: (..., 4)
    features = tf.stack([width, height, area, aspect], axis=-1)
    return features


# ============================================
# 2) Modelo logístico: y(x) = sigma(w^T x + b)
# ============================================

class LogisticCarDetector(tf.Module):
    """
    Modelo logístico binario para 'Car' vs 'no-Car'
    usando como entrada x ∈ R^4 (features del bbox).
    """

    def __init__(self, input_dim=4, name=None):
        super().__init__(name=name)

        # Parámetros del modelo:
        # w ∈ R^{D x 1}, b ∈ R
        self.w = tf.Variable(
            tf.random.normal(shape=(input_dim, 1), stddev=0.1),
            name="w"
        )
        self.b = tf.Variable(
            tf.zeros(shape=(1,)),
            name="b"
        )

    @tf.function
    def __call__(self, X):
        """
        X: tensor de shape (N, D) o (D,) con features:
           [w_bb, h_bb, area, aspect]

        Devuelve:
            y: probabilidad p(t=1 | x) con shape (N,)
        """
        X = tf.convert_to_tensor(X, dtype=tf.float32)

        # Asegurar shape (N, D)
        if len(X.shape) == 1:
            X = tf.expand_dims(X, axis=0)

        # logits = X w + b  → shape (N, 1)
        logits = tf.matmul(X, self.w) + self.b

        # y = sigma(logits) ∈ (0,1)
        y = tf.sigmoid(logits)  # shape (N, 1)

        # Devolver como vector (N,)
        return tf.squeeze(y, axis=-1)


# ============================================
# 3) Ejemplo de uso solo del modelo matemático
#    (sin entrenamiento todavía)
# ============================================

if __name__ == "__main__":
    # Ejemplo: un solo bounding box
    x1, y1, x2, y2 = 712.40, 143.00, 810.73, 307.92

    # 1) Construir features x = [w_bb, h_bb, area, aspect]
    x_feat = kitti_bbox_features(x1, y1, x2, y2)  # shape (4,)

    # 2) Crear el modelo
    model = LogisticCarDetector(input_dim=4)

    # 3) Forward: probabilidad de que esto sea un "Car"
    y_prob = model(x_feat)  # shape (), probabilidad en [0,1]

    print("Features:", x_feat.numpy())
    print("p(t=1 | x) =", float(y_prob))


Features: [9.8329956e+01 1.6492001e+02 1.6216578e+04 5.9622818e-01]
p(t=1 | x) = 1.0
